<a href="https://colab.research.google.com/github/Vahid-Kh/NLP/blob/main/gpt_dev_230222.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [5]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://github.com/Vahid-Kh/NLP/blob/da12adf98d303843e554adc71a4bd9ec1814bd93/test.txt

--2023-02-22 13:04:24--  https://github.com/Vahid-Kh/NLP/blob/da12adf98d303843e554adc71a4bd9ec1814bd93/test.txt
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘test.txt.1’

test.txt.1              [ <=>                ] 420.47K  --.-KB/s    in 0.04s   

2023-02-22 13:04:24 (11.4 MB/s) - ‘test.txt.1’ saved [430562]



In [8]:
# read it in to inspect it
with open('/content/test.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [9]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  4042155


In [12]:
# let's look at the first 1000 characters
print(text[:1000])
import re
#text = re.sub(r'\W+', ' ', text)
chars_to_drop = "ª¼ÂÃÎâ¡¢¤¥§¨©®°¶ˆ˜’‚“”†‰€™"
pattern = "[" + re.escape(chars_to_drop) + "]"
text = re.sub(pattern, "", text)


description
HUMAN NECESSITIES
AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTING; TRAPPING; FISHING
SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS, DETAILS, OR ACCESSORIES OF AGRICULTURAL MACHINES OR IMPLEMENTS, IN GENERAL 
Hand tools 
Spades; Shovels
with teeth
Hoes; Hand cultivators
with a single blade
with two or more blades
with blades provided with teeth
with teeth only
Tools for uprooting weeds
Tong-like tools
Combinations of different kinds of hand tools
Attaching the blades or the like to handles ; Interchangeable or adjustable blades
for treating meadows or lawns
Ploughs
Ploughs with fixed plough-shares
Man-driven ploughs
Animal-drawn ploughs
without alternating possibility, i.e. incapable of making an adjacent furrow on return journey
Swing ploughs
Trussed-beam ploughs; Single-wheel ploughs
Two-wheel beam ploughs
Frame ploughs
Alternating ploughs, i.e. capable of making an adjacent furrow on return journey
Turn-wrest ploughs
Balance ploughs
with parallel plough units used alter

In [13]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 "%'()+,-./0123456789:;<=>ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz 
82


In [14]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[62, 63, 63, 1, 74, 62, 59, 72, 59]
hii there


In [15]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([4040531]) torch.int64
tensor([58, 59, 73, 57, 72, 63, 70, 74, 63, 69, 68,  0, 34, 47, 39, 27, 40,  1,
        40, 31, 29, 31, 45, 45, 35, 46, 35, 31, 45,  0, 27, 33, 44, 35, 29, 47,
        38, 46, 47, 44, 31, 23,  1, 32, 41, 44, 31, 45, 46, 44, 51, 23,  1, 27,
        40, 35, 39, 27, 38,  1, 34, 47, 45, 28, 27, 40, 30, 44, 51, 23,  1, 34,
        47, 40, 46, 35, 40, 33, 23,  1, 46, 44, 27, 42, 42, 35, 40, 33, 23,  1,
        32, 35, 45, 34, 35, 40, 33,  0, 45, 41, 35, 38,  1, 49, 41, 44, 37, 35,
        40, 33,  1, 35, 40,  1, 27, 33, 44, 35, 29, 47, 38, 46, 47, 44, 31,  1,
        41, 44,  1, 32, 41, 44, 31, 45, 46, 44, 51, 23,  1, 42, 27, 44, 46, 45,
         8,  1, 30, 31, 46, 27, 35, 38, 45,  8,  1, 41, 44,  1, 27, 29, 29, 31,
        45, 45, 41, 44, 35, 31, 45,  1, 41, 32,  1, 27, 33, 44, 35, 29, 47, 38,
        46, 47, 44, 27, 38,  1, 39, 27, 29, 34, 35, 40, 31, 45,  1, 41, 44,  1,
        35, 39, 42, 38, 31, 39, 31, 40, 46, 45,  8,  1, 35, 40,  1, 33, 31, 40,
      

In [16]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [17]:
block_size = 8
train_data[:block_size+1]

tensor([58, 59, 73, 57, 72, 63, 70, 74, 63])

In [18]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([58]) the target: 59
when input is tensor([58, 59]) the target: 73
when input is tensor([58, 59, 73]) the target: 57
when input is tensor([58, 59, 73, 57]) the target: 72
when input is tensor([58, 59, 73, 57, 72]) the target: 63
when input is tensor([58, 59, 73, 57, 72, 63]) the target: 70
when input is tensor([58, 59, 73, 57, 72, 63, 70]) the target: 74
when input is tensor([58, 59, 73, 57, 72, 63, 70, 74]) the target: 63


In [19]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[66, 63, 57,  1, 72, 63, 68, 61],
        [74, 63, 58, 59, 73,  1, 69, 72],
        [55, 74, 63, 69, 68,  8,  1, 58],
        [68, 59, 73, 59,  1, 69, 78, 63]])
targets:
torch.Size([4, 8])
tensor([[63, 57,  1, 72, 63, 68, 61, 73],
        [63, 58, 59, 73,  1, 69, 72,  1],
        [74, 63, 69, 68,  8,  1, 58, 59],
        [59, 73, 59,  1, 69, 78, 63, 58]])
----
when input is [66] the target: 63
when input is [66, 63] the target: 57
when input is [66, 63, 57] the target: 1
when input is [66, 63, 57, 1] the target: 72
when input is [66, 63, 57, 1, 72] the target: 63
when input is [66, 63, 57, 1, 72, 63] the target: 68
when input is [66, 63, 57, 1, 72, 63, 68] the target: 61
when input is [66, 63, 57, 1, 72, 63, 68, 61] the target: 73
when input is [74] the target: 63
when input is [74, 63] the target: 58
when input is [74, 63, 58] the target: 59
when input is [74, 63, 58, 59] the target: 73
when input is [74, 63, 58, 59, 73] the target: 1
when input is [

In [20]:
print(xb) # our input to the transformer

tensor([[66, 63, 57,  1, 72, 63, 68, 61],
        [74, 63, 58, 59, 73,  1, 69, 72],
        [55, 74, 63, 69, 68,  8,  1, 58],
        [68, 59, 73, 59,  1, 69, 78, 63]])


In [21]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 82])
tensor(4.8566, grad_fn=<NllLossBackward0>)

3/;4mKMOJyiCoE)9FzPPVX1fdFJaYJ%1PSUFhV<VQTffdeti;JTfX':%'v8fH O.2G,NsQ+Y"(o<gghJBXONb6:yBExdq+Z'gr'b


In [22]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [23]:
batch_size = 32
for steps in range(100): # increase number of steps for good results... 
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.808898448944092


In [24]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


EjwF)d8J/t02UYf[53A<uM%m6  OVaz%[8UTn1 AW<]JHGg:NW;Jo<2ukef(/tg7XTUX+NFn] 5Qa ,P[XOsaDOUR=QT<D)eRb9LG012r]"EJvGq]kYGn70teW;B/b;3Q8dAGlv"7;jMGDk ULGt=w:)Llkk3WYhZVJf.=8OItxE1p3 mSXO8G9."[Hr[v"(Y6rn6/H[SKP4ak=7Oq
E90jy=E[H[Bz cNm.=1-hlkv"E1u4OJ< '6rnT8MuOI4tei(nT]DwRtZvvQ)Nk5Ip%R]ga )9/GnEt;SYp97>Oi+kuxW GDk5HTlk=VUTK1mK-kJkvPy>Da:h(/<DwuIML-;gv"Rh>2bTS9Ai,-R+ew:<'QtND9.lB1U."fy0qCteJZm
r'Jo<u96
E)beG2U,]o7Y=O1C9sL"D[V,"qOblcy5HKGgTC- O02CbWYxYU5Bv/If1.d A
uRPPoGlk)J<u4: r[H%w9V21v/ND,u B/M+9'MYFL


## The mathematical trick in self-attention

In [25]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [26]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [27]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [28]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [29]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [30]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [31]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [32]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [33]:
k.var()

tensor(1.0449)

In [34]:
q.var()

tensor(1.0700)

In [35]:
wei.var()

tensor(1.0918)

In [36]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [37]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [38]:
class LayerNorm1d: # (used to be BatchNorm1d)
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [39]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [40]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [41]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [45]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)



# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.211922 M parameters
step 0: train loss 4.5792, val loss 4.5725
step 100: train loss 2.6542, val loss 2.6581
step 200: train loss 2.4859, val loss 2.5204
step 300: train loss 2.3945, val loss 2.4186
step 400: train loss 2.3108, val loss 2.3587
step 500: train loss 2.2274, val loss 2.2479
step 600: train loss 2.1409, val loss 2.1865
step 700: train loss 2.0844, val loss 2.1373
step 800: train loss 2.0314, val loss 2.0777
step 900: train loss 1.9792, val loss 2.0405
step 1000: train loss 1.9334, val loss 2.0108
step 1100: train loss 1.9065, val loss 1.9696
step 1200: train loss 1.8758, val loss 1.9437
step 1300: train loss 1.8205, val loss 1.8868
step 1400: train loss 1.7995, val loss 1.8576
step 1500: train loss 1.7694, val loss 1.8278
step 1600: train loss 1.7437, val loss 1.8134
step 1700: train loss 1.7478, val loss 1.7913
step 1800: train loss 1.7135, val loss 1.7797
step 1900: train loss 1.6965, val loss 1.7716
step 2000: train loss 1.6728, val loss 1.7480
step 2100: train loss 1.

In [47]:
import torch
text_prompt = " related superheat sensing                     "
data = torch.tensor(encode(text_prompt), dtype=torch.long)
ix =torch.randint(len(data)-block_size,(batch_size,))
x = torch.stack([data[i:i+block_size] for i in ix])
print(decode(m.generate(x, max_new_tokens=1000)[0].tolist()))

perheat sensing                 obtainete withmiter, e.g. Coloth
Cords Skeleton
Thickines
with is and the tright or hetero atoms
having blood, or colloveled withmils fluid, e.g. for using GAS anthrost plutral, aromissilar container
contacts; Croating or position fhoces overlocyes
with edepring michanble tensitutith oxygen or uposition 
operating filters
Dirous
Boltant-Adhesive, e.g. parts, not oxygen-radised compacity, e.g. locks
Preceptyther containing over groups; Arrangements to perior ploughts
awing devices for inces being the flow; Oil
Sucoused, e.g. hydrococyclonic rings
with high a charge to charide ight-loders for this torbity of fuel
Posbitalcyhydric contautoght of electrophileve or plughers
Trimpersible movable both raw, section, or or socyclic groups air anims or such pressure, e.g. of wire heter in
hydrons
companuming devices for to furns for mowererls, not havided from other their curtterce
containing vehicle arranged rings
Processes 
Electrical to te expandcent a the frox